### Author: Parneet
#### Blood brain barrier permeability prediction using dataset provided in the paper-

( https://www.mdpi.com/1420-3049/26/24/7428 )


In [ ]:
#installing rdkit
# !pip install rdkit-pypi

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, f_classif

# https://www.rdkit.org/
#https://github.com/rdkit/rdkit
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors



In [ ]:
#loading the dataet
data = pd.read_csv("bbbp_blood.csv")
print(data.shape)
data.head()

### Generate Canonical SMILES-
#### Why?
#### Read here- [Tutorial to SMILES and canonical SMILES explained with examples](https://luis-vollmers.medium.com/tutorial-to-smiles-and-canonical-smiles-explained-with-examples-fbc8a46ca29f)

In [ ]:
# There might be one or more valid SMILES that can represent one compound
def canonical_smiles(smiles):
    mols = [Chem.MolFromSmiles(smi) for smi in smiles]
    smiles = [Chem.MolToSmiles(mol) for mol in mols]
    return smiles

In [ ]:
# Canonical SMILES for main dataset
Canon_SMILES = canonical_smiles(data.smiles)
len(Canon_SMILES)

In [ ]:
# Concat the smiles in the dataframe
data['SMILES'] = Canon_SMILES
data = data.drop(['smiles'], axis=1)
data.head()

In [ ]:
# Check if there are any duplicate smiles
# Create a list for duplicate smiles
duplicates_smiles = data[data['SMILES'].duplicated()]['SMILES'].values
len(duplicates_smiles)
#no duplicate smiles are present in the dataset

### Generate molecular descriptors using RDkit




In [ ]:
# Get list of all RDKit descriptor names
descriptor_names = [desc[0] for desc in Descriptors._descList]
# Create a calculator for the descriptors
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)
# Function to compute descriptors from SMILES
def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return list(calculator.CalcDescriptors(mol))
    else:
        return [None] * len(descriptor_names)

# Apply function to the dataset
df_descriptors = data["SMILES"].apply(compute_descriptors)
# Convert list of descriptor values into a DataFrame
df_descriptors = pd.DataFrame(df_descriptors.tolist(), columns=descriptor_names)
# Concatenate original dataset with descriptor values
df_final = pd.concat([data, df_descriptors], axis=1)

print("RDKit descriptors generated and saved!")


In [ ]:
df_final #full dataframe with original columns and rdkit descriptors

In [ ]:
df_final.isna().sum().sum() ##checking for any na values

In [ ]:
#check if any columns have all zero values: safe to remove
zero_cols = df_final.columns[(df_final == 0).all()]
print("Columns with zero values-")
print(zero_cols)
print("Original Shape ")
print(df_final.shape)
print("======")
if len(zero_cols) > 0:
    print(f"Removing {len(zero_cols)} columns with all zero values: {list(zero_cols)}")
    df_final = df_final.drop(columns=zero_cols)
print("Shape after rem columms")
print(df_final.shape)

### ML: Classification using Random Forest

In [ ]:
#extracting the labels into y variable
y = df_final['p_np']
y

In [ ]:
print(y.value_counts()) #counting 0 and 1 labels

In [ ]:
#features to be trained using the ML models: X
X = df_final.drop(['num',	'name',	'p_np',	'SMILES'], axis = 1)
X

In [ ]:
X.shape

In [ ]:
X.describe()

In [ ]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.30, random_state=0)
#split ratio:
#train:test = 70:30

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
#feature Scaling : Apply MinMaxScaler to scale the descriptors between 0 and 1.
st_x= MinMaxScaler()
X_train= st_x.fit_transform(X_train)   #call fit_transform() on train data
X_test= st_x.transform(X_test)         #call transform() on test data

# X_train

In [ ]:
# Random forest model
from sklearn.ensemble import RandomForestClassifier
model_RF = RandomForestClassifier()
model_RF.fit(X_train, y_train)

In [ ]:
#Predicting the test set result
y_pred_RF= model_RF.predict(X_test)
y_pred_RF;

In [ ]:
###Evaluating the Model:

print("Results for Random Forest Model:\n")

# Accuracy Score
print('Accuracy Score:', accuracy_score(y_test, y_pred_RF))
print()

# Confusion Matrix
results = confusion_matrix(y_test, y_pred_RF)
print('Confusion Matrix:')
print(results)
print()

# Precision
print('Precision Score:', precision_score(y_test, y_pred_RF))
print()

# Recall
print('Recall Score:', recall_score(y_test, y_pred_RF))
print()

# F1 Score
print('F1 Score:', f1_score(y_test, y_pred_RF))
print()

# ROC-AUC Score (for binary classification)
print('ROC-AUC Score:', roc_auc_score(y_test, y_pred_RF))
print()

# Classification Report
print('Classification Report:')
print(classification_report(y_test, y_pred_RF))


## Feature Importance: RF Importance & SKBest


In [ ]:
#RANDOM FOREST IMPORTANCE
# Get feature names and importances
features = X.columns
importances = model_RF.feature_importances_

# Sort feature importances in descending order
indices = np.argsort(importances)[-10:]  # Select top 10 features

# Plot
plt.figure(figsize=(10, 6))
plt.title('Top 10 Features: RF Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center', height=0.5)
plt.yticks(range(len(indices)), [features[i] for i in indices], fontsize=10)
plt.xlabel('Relative Importance')
plt.savefig("feature_importance_RF.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# SELECTkBEST
k = 10  # Number of top features to select
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X, y)

# Get selected feature scores and names
scores = selector.scores_
selected_indices = np.argsort(scores)[-k:]  # Top k features

# Get feature names and their scores
selected_features = [X.columns[i] for i in selected_indices]
selected_scores = [scores[i] for i in selected_indices]

# Plot top 10 selected features
plt.figure(figsize=(10, 6))
plt.title('Top 10 Features (SelectKBest)')
plt.barh(range(k), selected_scores, color='b', align='center', height=0.5)
plt.yticks(range(k), selected_features, fontsize=10)
plt.xlabel('Feature Score')
plt.savefig("feature_importance_skb.png", dpi=300, bbox_inches='tight')
plt.show()
